## Capston Project - New York Chinese Restaurants

### Problem

New York City (NYC), often called the City of New York or simply New York (NY), is the most populous city in the United States. With an estimated 2018 population of 8,398,748 distributed over about 302.6 square miles (784 km2), New York is also the most densely populated major city in the United States. Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass. With almost 20 million people in its metropolitan statistical area and approximately 23 million in its combined statistical area, it is one of the world's most populous megacities. New York City has been described as the cultural, financial, and media capital of the world, significantly influencing commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports. Home to the headquarters of the United Nations. New York is an important center for international diplomacy.

Besides to what I just mentioned before New York is one of the most cosmopolitan city in the whole world. There a lot of people of different backgrounds, and there a lot of different kind of restaurants around the city. In the matter of cuisine, there are a lot of chinese, thai, japanese and korean restaurants, and for our concerns we need to choose a new neighbourhood for a new chinese restaurant that we want build. 

### Data 

For this project we need the following data :

* New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.
 * Data source : https://cocl.us/new_york_dataset
 * Description : This data set contains the required information. And we will use this data set to explore various neighborhoods of New York city.
* Chinese restaurants in each neighborhood of New York city.
 * Data source : Fousquare API
 * Description : Using this API we will get all the venues in each neighborhood. We can filter these venues to get only Asian restaurants.
* GeoSpace data
 * Data source : https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm
 * Description : By using this geo space data we will get the New York Borough boundaries that will help us visualize choropleth map.


### Methodology 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

ImportError: cannot import name 'OrderedDict'

  Borough Neighborhood   Latitude  Longitude
0   Bronx    Wakefield  40.894705 -73.847201
1   Bronx   Co-op City  40.874294 -73.829939
2   Bronx  Eastchester  40.887556 -73.827806
3   Bronx    Fieldston  40.895437 -73.905643
4   Bronx    Riverdale  40.890834 -73.912585

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
CLIENT_ID = 'N4HVBCK0QG4P00NVLAXNHOIGO3VP2GRCI0HGDB0OFUVKPJP1' # your Foursquare ID
CLIENT_SECRET = 'T21YISJIBUJZHRW1AGXDHOFIVWTPJYTQ2W1MCBEC2QOLEDA2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N4HVBCK0QG4P00NVLAXNHOIGO3VP2GRCI0HGDB0OFUVKPJP1
CLIENT_SECRET:T21YISJIBUJZHRW1AGXDHOFIVWTPJYTQ2W1MCBEC2QOLEDA2


In [14]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
# type your answer here

ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [21]:
ny_venues.groupby('Neighborhood').count()

ImportError: cannot import name 'OrderedDict'

                           Neighborhood Latitude  Neighborhood Longitude  \
Neighborhood                                                               
Allerton                                      31                      31   
Annadale                                       9                       9   
Arden Heights                                  6                       6   
Arlington                                      7                       7   
Arrochar                                      21                      21   
Arverne                                       18                      18   
Astoria                                      100                     100   
Astoria Heights                               11                      11   
Auburndale                                    19                      19   
Bath Beach                                    45                      45   
Battery Park City                             87                      87   
Bay Ridge   

In [23]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 430 uniques categories.


In [28]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]


In [29]:
ny_onehot.shape

(10243, 430)

In [30]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()


In [31]:
ny_grouped.shape

(301, 430)

In [32]:
num_top_venues = 10

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                 venue  freq
0          Pizza Place  0.19
1     Department Store  0.06
2        Deli / Bodega  0.06
3          Supermarket  0.06
4   Chinese Restaurant  0.06
5          Bus Station  0.06
6           Playground  0.03
7  Fried Chicken Joint  0.03
8          Gas Station  0.03
9   Spanish Restaurant  0.03


----Annadale----
                   venue  freq
0            Pizza Place  0.22
1    American Restaurant  0.11
2          Train Station  0.11
3             Sports Bar  0.11
4                  Diner  0.11
5             Restaurant  0.11
6           Dance Studio  0.11
7                    Pub  0.11
8                   Park  0.00
9  Outdoors & Recreation  0.00


----Arden Heights----
                   venue  freq
0               Pharmacy  0.17
1          Moving Target  0.17
2            Coffee Shop  0.17
3                   Pool  0.17
4            Pizza Place  0.17
5             Playground  0.17
6            Yoga Studio  0.00
7  Outdoors & Recreation  0.00
8

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)


In [35]:
# set number of clusters
kclusters = 10

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 8, 3, 6, 6, 3, 6, 6], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# add clustering labels
ny_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

 # check the last columns!


In [49]:
ny_merged['Cluster Labels'] = ny_merged['Cluster Labels'].dropna()

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'],ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [52]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

        Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0          Wakefield              Pharmacy        Ice Cream Shop   
2        Eastchester  Caribbean Restaurant         Deli / Bodega   
9     Williamsbridge                   Bar            Soup Place   
41         Olinville  Caribbean Restaurant      Basketball Court   
56     East Flatbush        Hardware Store      Department Store   
73     Starrett City  Caribbean Restaurant              Pharmacy   
74          Canarsie    Chinese Restaurant        Shipping Store   
75         Flatlands              Pharmacy   Fried Chicken Joint   
78      Coney Island      Baseball Stadium  Caribbean Restaurant   
165       St. Albans  Caribbean Restaurant                Market   
168  Cambria Heights  Caribbean Restaurant        Cosmetics Shop   
188        Laurelton  Caribbean Restaurant        Cosmetics Shop   
223            Rugby  Caribbean Restaurant                  Bank   
257     Howland Hook                   NaN      

In [53]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

    Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
207   Port Ivory                   Bar   Filipino Restaurant   
212      Oakwood         Women's Store                   Bar   

     3rd Most Common Venue  4th Most Common Venue 5th Most Common Venue  \
207  Entertainment Service   Ethiopian Restaurant           Event Space   
212                 Lawyer  Entertainment Service  Ethiopian Restaurant   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
207               Exhibit            Eye Doctor               Factory   
212           Event Space               Exhibit            Eye Doctor   

    9th Most Common Venue 10th Most Common Venue  
207    Falafel Restaurant                   Farm  
212               Factory     Falafel Restaurant  

In [54]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

         Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
193        Brookville         Deli / Bodega         Women's Store   
202       Grymes Hill               Dog Run         Deli / Bodega   
206  Mariner's Harbor    Italian Restaurant         Deli / Bodega   

    3rd Most Common Venue  4th Most Common Venue 5th Most Common Venue  \
193   Filipino Restaurant  Entertainment Service  Ethiopian Restaurant   
202         Women's Store    Filipino Restaurant  Ethiopian Restaurant   
206   Filipino Restaurant  Entertainment Service  Ethiopian Restaurant   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
193           Event Space               Exhibit            Eye Doctor   
202           Event Space               Exhibit            Eye Doctor   
206           Event Space               Exhibit            Eye Doctor   

    9th Most Common Venue 10th Most Common Venue  
193               Factory     Falafel Restaurant  
202               Factory     F

In [55]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

          Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
7             Woodlawn           Pizza Place                   Pub   
28         Throgs Neck    Italian Restaurant   American Restaurant   
32            Van Nest           Pizza Place         Deli / Bodega   
34             Belmont    Italian Restaurant           Pizza Place   
39      Edgewater Park    Italian Restaurant         Deli / Bodega   
72       East New York         Deli / Bodega            Food Truck   
89          Ocean Hill         Deli / Bodega            Playground   
134             Corona    Mexican Restaurant    Italian Restaurant   
148   South Ozone Park                  Park         Deli / Bodega   
149      College Point         Deli / Bodega                Bakery   
150         Whitestone       Bubble Tea Shop          Dance Studio   
154         Douglaston                  Bank         Deli / Bodega   
159          Briarwood            Playground         Deli / Bodega   
174         Beechhur

In [56]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

            Neighborhood      1st Most Common Venue    2nd Most Common Venue  \
1             Co-op City                Bus Station     Fast Food Restaurant   
8                Norwood                Pizza Place                     Park   
13          Bedford Park                      Diner       Chinese Restaurant   
14    University Heights                Pizza Place                   Bakery   
15        Morris Heights                       Park  Health & Beauty Service   
17          East Tremont                Pizza Place        Mobile Phone Shop   
19          High  Bridge                   Pharmacy              Pizza Place   
20               Melrose                Pizza Place            Grocery Store   
21            Mott Haven                        Gym            Grocery Store   
33           Morris Park                Pizza Place             Burger Joint   
35        Spuyten Duyvil                       Park                     Bank   
36       North Riverdale                

In [44]:
ny_merged.loc[ny_merged['Cluster Labels'] == 6, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

                  Neighborhood            1st Most Common Venue  \
3                    Fieldston                   Medical School   
4                    Riverdale                             Park   
5                  Kingsbridge                      Pizza Place   
6                  Marble Hill                   Sandwich Place   
10                  Baychester                       Donut Shop   
11              Pelham Parkway               Italian Restaurant   
12                 City Island                  Harbor / Marina   
16                     Fordham             Fast Food Restaurant   
18                  West Farms                      Bus Station   
22                 Port Morris                     Cupcake Shop   
23                    Longwood             Fast Food Restaurant   
24                 Hunts Point                        Juice Bar   
25                  Morrisania                   Discount Store   
26                   Soundview               Chinese Restauran

In [46]:
ny_merged.loc[ny_merged['Cluster Labels'] == 7, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

     Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
286  Sandy Ground              Bus Stop                Market   
305     Fox Hills              Bus Stop        Sandwich Place   

    3rd Most Common Venue 4th Most Common Venue  5th Most Common Venue  \
286          Intersection   Filipino Restaurant  Entertainment Service   
305         Women's Store   Filipino Restaurant   Ethiopian Restaurant   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
286  Ethiopian Restaurant           Event Space               Exhibit   
305           Event Space               Exhibit            Eye Doctor   

    9th Most Common Venue 10th Most Common Venue  
286            Eye Doctor                Factory  
305               Factory     Falafel Restaurant  

In [47]:
ny_merged.loc[ny_merged['Cluster Labels'] == 8, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

        Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
77   Manhattan Beach              Bus Stop       Harbor / Marina   
198     New Brighton              Bus Stop                  Park   
224        Park Hill              Bus Stop                 Hotel   
227        Arlington              Bus Stop          Intersection   
232    Midland Beach                 Beach         Deli / Bodega   
245       Bloomfield                  Park     Recreation Center   
256    Randall Manor              Bus Stop           Pizza Place   
285      Willowbrook              Bus Stop         Deli / Bodega   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
77                   Café        Sandwich Place                 Beach   
198         Deli / Bodega            Playground        Discount Store   
224  Gym / Fitness Center           Coffee Shop    Athletics & Sports   
227         Deli / Bodega           Coffee Shop         Boat or Ferry   
232              Bus S

In [48]:
ny_merged.loc[ny_merged['Cluster Labels'] == 9, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

ImportError: cannot import name 'OrderedDict'

     Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
85       Sea Gate                 Beach           Bus Station   
172  Breezy Point                 Trail                 Beach   
179      Neponsit                 Beach                Lounge   
204   South Beach                 Beach                  Pier   
302       Hammels                 Beach            Shoe Store   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
85                    Spa          Home Service         Women's Store   
172   Monument / Landmark         Women's Store                 Field   
179         Women's Store    English Restaurant  Ethiopian Restaurant   
204         Deli / Bodega    Athletics & Sports         Women's Store   
302              Bus Stop  Fast Food Restaurant               Dog Run   

    6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
85                  Field  Ethiopian Restaurant           Event Space   
172  Ethiopian Restauran